# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 26: Aprendizaje Supervisado: PCA</center></h1>

En este notebook aplicaremos las herramientas implementadas en las librerías de Python `sklearn` para realizar análisis de componentes principales.

## Caso de Estudio: Análisis de Base de Datos de Cancer de Mama

El uso de PCA permite analizar conjuntos de datos multidimensionales y encontrar la información relevante que tienen. 

### Descripción del Dataset

Usaremos la base de datos de cáncer de mama creada por UC Irvine. Pueden ver los detalles de la base en el siguiente sitio: https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic

### Objetivo

El objetivo de esta actividad es entender los factores relevantes en el cáncer de mama según lo que aparece en la base de datos.

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import sklearn as sk

## 1. Datos a analizar

La base de datos la podemos importar directamente del repositorio de `sklearn` de la siguiente forma:

In [ ]:
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()
breast_data = breast.data

Exploremos los datos ahora.

In [ ]:
breast_data.shape

In [ ]:
breast_data

In [ ]:
breast_labels = breast.target
breast_labels.shape

Ahora conectemos todos los datos para armar nuestra base de datos a usar posteriormente.

In [ ]:
labels = np.reshape(breast_labels,(569,1))

In [ ]:
final_breast_data = np.concatenate([breast_data,labels],axis=1)

In [ ]:
final_breast_data.shape

In [ ]:
breast_dataset = pd.DataFrame(final_breast_data)
breast_dataset.head(5)

In [ ]:
features = breast.feature_names
features

In [ ]:
features_labels = np.append(features,'label')

In [ ]:
breast_dataset.columns = features_labels

In [ ]:
breast_dataset.head(10)

In [ ]:
breast_dataset['label'].replace(0, 'Benign', inplace=True)
breast_dataset['label'].replace(1, 'Malignant', inplace=True)
breast_dataset.tail(5)

## 2. EDA

Partamos buscando conectar la columna de `label` con causas que puedan ser relevantes. Para ello comenzamos con un EDA simple.

In [ ]:
sns.pairplot(breast_dataset, y_vars='label')

In [ ]:
plt.figure()
plt.figure(figsize=(10,10))
plt.xticks(fontsize=12)
plt.yticks(fontsize=14)
plt.xlabel('Mean Radius', fontsize=20)
plt.ylabel('Mean Texture', fontsize=20)
plt.title("Relaciones entre características", fontsize=20)
targets = ['Benign', 'Malignant']
colors = ['r', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = breast_dataset['label'] == target
    plt.scatter(breast_dataset[breast_dataset['label']==target]['mean radius']
               , breast_dataset[breast_dataset['label']==target]['mean texture'], c = color, s = 50)
plt.legend(targets,prop={'size': 15})

## 3. PCA

Veamos ahora cómo usar PCA para encontrar información relevante en las diferentes dimensiones de los datos.

Debemos comenzar escalando los datos como hemos comentado en clases.

In [ ]:
from sklearn.preprocessing import StandardScaler

x = breast_dataset.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [ ]:
x.shape

In [ ]:
np.mean(x),np.std(x)

In [ ]:
feat_cols = ['feature'+str(i) for i in range(x.shape[1])]
normalised_breast = pd.DataFrame(x,columns=feat_cols)

In [ ]:
normalised_breast.tail(10)

Ahora transformemeos estos datos que están en 30 dimensiones, a una proyección en 2 dimensiones usando PCA.

In [ ]:
from sklearn.decomposition import PCA

pca_breast = PCA(n_components=2)
principalComponents_breast = pca_breast.fit_transform(x)

In [ ]:
principal_breast_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
principal_breast_Df.tail()

In [ ]:
print('Explained variability per principal component: {}'.format(pca_breast.explained_variance_ratio_))

In [ ]:
plt.figure()
plt.figure(figsize=(10,10))
plt.xticks(fontsize=12)
plt.yticks(fontsize=14)
plt.xlabel('Componente Principal 1', fontsize=20)
plt.ylabel('Componente Principal 2', fontsize=20)
plt.title("Análisis de Componentes Principales", fontsize=20)
targets = ['Benign', 'Malignant']
colors = ['r', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = breast_dataset['label'] == target
    plt.scatter(principal_breast_Df.loc[indicesToKeep, 'principal component 1']
               , principal_breast_Df.loc[indicesToKeep, 'principal component 2'], c = color, s = 50)
plt.legend(targets,prop={'size': 15})